In [96]:
from torch_geometric.datasets import ZINC
from torch_geometric.loader import DataLoader

root = "datasets/ZINC"
dataset = ZINC(root, subset=True, split="train")
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

for step, batch in enumerate(train_loader):
    if step == 0:
        # print(batch.batch)
        # print(batch.batch.shape)
        # print(batch.x.shape)
        # print(batch.x)
        data = batch
        # print(batch.subgraph_idx_batch)
        # print(batch)
        break
print(data)
data = add_full_rrwp(data)
print(data)
expanded_edge_val_dummy = torch.zeros(data.rrwp_index.shape[1], dtype=torch.int64)
expanded_edge_index, expanded_edge_val = torch_sparse.coalesce(torch.cat([data.edge_index, data.rrwp_index], dim=1), 
                    torch.cat([data.edge_attr, expanded_edge_val_dummy], dim=0), data.num_nodes, data.num_nodes,
                    op="add")
print(expanded_edge_index.shape)
print(expanded_edge_val.shape)
data.expanded_edge_index = expanded_edge_index
data.expanded_edge_attr = expanded_edge_val
print(data)

DataBatch(x=[705, 1], edge_index=[2, 1522], edge_attr=[1522], y=[32], batch=[705], ptr=[33])
DataBatch(x=[705, 1], edge_index=[2, 1522], edge_attr=[1522], y=[32], batch=[705], ptr=[33], rrwp=[705, 8], rrwp_index=[2, 12605], rrwp_val=[12605, 8], log_deg=[705], deg=[705])
torch.Size([2, 12605])
torch.Size([12605])
DataBatch(x=[705, 1], edge_index=[2, 1522], edge_attr=[1522], y=[32], batch=[705], ptr=[33], rrwp=[705, 8], rrwp_index=[2, 12605], rrwp_val=[12605, 8], log_deg=[705], deg=[705], expanded_edge_index=[2, 12605], expanded_edge_attr=[12605])


In [30]:
from typing import Union, Any, Optional
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric as pyg
from torch_geometric.data import Data, HeteroData
from torch_geometric.transforms import BaseTransform
from torch_scatter import scatter, scatter_add, scatter_max

from torch_geometric.graphgym.config import cfg

from torch_geometric.utils import (
    get_laplacian,
    get_self_loop_attr,
    to_scipy_sparse_matrix,
)
import torch_sparse
from torch_sparse import SparseTensor

def add_node_attr(data: Data, value: Any,
                  attr_name: Optional[str] = None) -> Data:
    if attr_name is None:
        if 'x' in data:
            x = data.x.view(-1, 1) if data.x.dim() == 1 else data.x
            data.x = torch.cat([x, value.to(x.device, x.dtype)], dim=-1)
        else:
            data.x = value
    else:
        data[attr_name] = value

    return data


@torch.no_grad()
def add_full_rrwp(data,
                  walk_length=8,
                  attr_name_abs="rrwp", # name: 'rrwp'
                  attr_name_rel="rrwp", # name: ('rrwp_idx', 'rrwp_val')
                  add_identity=True,
                  spd=False,
                  **kwargs
                  ):
    device=data.edge_index.device
    ind_vec = torch.eye(walk_length, dtype=torch.float, device=device)
    num_nodes = data.num_nodes
    edge_index, edge_weight = data.edge_index, data.edge_weight

    adj = SparseTensor.from_edge_index(edge_index, edge_weight,
                                       sparse_sizes=(num_nodes, num_nodes),
                                       )

    # Compute D^{-1} A:
    deg = adj.sum(dim=1)
    deg_inv = 1.0 / adj.sum(dim=1)
    deg_inv[deg_inv == float('inf')] = 0
    adj = adj * deg_inv.view(-1, 1)
    adj = adj.to_dense()

    pe_list = []
    i = 0
    if add_identity:
        pe_list.append(torch.eye(num_nodes, dtype=torch.float))
        i = i + 1

    out = adj
    pe_list.append(adj)

    if walk_length > 2:
        for j in range(i + 1, walk_length):
            out = out @ adj
            pe_list.append(out)

    pe = torch.stack(pe_list, dim=-1) # n x n x k

    abs_pe = pe.diagonal().transpose(0, 1) # n x k

    rel_pe = SparseTensor.from_dense(pe, has_value=True)
    rel_pe_row, rel_pe_col, rel_pe_val = rel_pe.coo()
    rel_pe_idx = torch.stack([rel_pe_row, rel_pe_col], dim=0)

    if spd:
        spd_idx = walk_length - torch.arange(walk_length)
        val = (rel_pe_val > 0).type(torch.float) * spd_idx.unsqueeze(0)
        val = torch.argmax(val, dim=-1)
        rel_pe_val = F.one_hot(val, walk_length).type(torch.float)
        abs_pe = torch.zeros_like(abs_pe)

    data = add_node_attr(data, abs_pe, attr_name=attr_name_abs)
    data = add_node_attr(data, rel_pe_idx, attr_name=f"{attr_name_rel}_index")
    data = add_node_attr(data, rel_pe_val, attr_name=f"{attr_name_rel}_val")
    data.log_deg = torch.log(deg + 1)
    data.deg = deg.type(torch.long)

    return data

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn

from torch import Tensor
from torch_geometric.typing import OptPairTensor
from torch_geometric.graphgym.models.layer import LayerConfig
from torch_geometric.graphgym.register import register_layer
from torch_geometric.nn import Linear as Linear_pyg



class FilteredGINELayer(pyg_nn.conv.MessagePassing):
    """GINEConv Layer with Filtration implementation.

    Modified torch_geometric.nn.conv.GINECon layer to perform learned message scaling
    according to graph_encoding
    """
    
    def __init__(self, nn, eps=0., train_eps=False, edge_dim=None, **kwargs):
        kwargs.setdefault('aggr', 'add')
        super().__init__(**kwargs)
        self.nn = nn
        self.initial_eps = eps
        if train_eps:
            self.eps = torch.nn.Parameter(torch.Tensor([eps]))
        else:
            self.register_buffer('eps', torch.Tensor([eps]))
        if edge_dim is not None:
            if isinstance(self.nn, torch.nn.Sequential):
                nn = self.nn[0]
            if hasattr(nn, 'in_features'):
                in_channels = nn.in_features
            elif hasattr(nn, 'in_channels'):
                in_channels = nn.in_channels
            else:
                raise ValueError("Could not infer input channels from `nn`.")
            self.lin = pyg_nn.Linear(edge_dim, in_channels)
        else:
            self.lin = None
        self.reset_parameters()

    
    def reset_parameters(self):
        pyg_nn.inits.reset(self.nn)
        self.eps.data.fill_(self.initial_eps)
        if self.lin is not None:
            self.lin.reset_parameters()

    def forward(self, x, edge_index, edge_attr=None, mask=None, size=None):
        if isinstance(x, Tensor):
            x: OptPairTensor = (x, x)

        # propagate_type: (x: OptPairTensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, mask=mask)
        
        x_r = x[1]
        if x_r is not None:
            out += (1 + self.eps) * x_r

        return self.nn(out)
    
    def message(self, x_j, edge_attr, mask):
        if self.lin is None and x_j.size(-1) != edge_attr.size(-1):
            raise ValueError("Node and edge feature dimensionalities do not "
                    "match. Consider setting the 'edge_dim' "
                    "attribute of 'GINEConv'")
        print("x_j", x_j.shape)
        print("edge_attr: ", edge_attr.shape)
        if self.lin is not None:
            edge_attr = self.lin(edge_attr)
        return torch.einsum('i,ij->ij', mask, (x_j + edge_attr).relu())

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}(nn={self.nn})'
        

In [37]:
# data = add_full_rrwp(data)
# import torch_sparse
# rrwp_idx = data.rrwp_index
# rrwp_val = data.rrwp_val
# edge_index = data.edge_index
# edge_attr = torch.rand([edge_index.shape[1], rrwp_val.shape[1]])

# if edge_attr is None:
#     edge_attr = edge_index.new_zeros(edge_index.size(1), rrwp_val.size(1))
#     # zero padding for non-existing edges

# out_idx, out_val = torch_sparse.coalesce(
#     torch.cat([edge_index, rrwp_idx], dim=1),
#     torch.cat([edge_attr, rrwp_val], dim=0),
#     batch.num_nodes, batch.num_nodes,
#     op="add"
#         )

# print(rrwp_idx.shape)
# print(rrwp_val.shape)
# print(out_idx.shape)
# print(out_val.shape)

torch.Size([2, 13836])
torch.Size([13836, 8])
torch.Size([2, 13836])
torch.Size([13836, 8])


In [99]:
import torch_geometric.nn as pyg_nn
import torch.nn.functional as F

dim_in = 21
dim_out = 64
num_clusters = 5
mlp = nn.Sequential(pyg_nn.Linear(dim_in, dim_out), nn.ReLU(),
                    pyg_nn.Linear(dim_out, dim_out))
gineLayer = FilteredGINELayer(mlp)

encoding = torch.randn([data.rrwp_val.shape[0], num_clusters])
masks = F.softmax(encoding, dim=-1)
# print(masks.shape)
masks = torch.transpose(masks, 0, 1)
# print(masks.shape)
# print(masks[0].shape)
one_vec = torch.ones([masks.shape[1], 3])
# print(masks[0])
# print(torch.einsum('i,ij->ij', masks[0], one_vec))
h = torch.randn([data.x.shape[0], dim_in])
encoder = torch.nn.Embedding(num_embeddings=4, embedding_dim=dim_in)
print(data.expanded_edge_attr)
expanded_edge_attr = encoder(data.expanded_edge_attr)
x = gineLayer(h, data.expanded_edge_index, expanded_edge_attr, masks[0])

tensor([0, 1, 0,  ..., 0, 1, 0])
x_j torch.Size([12605, 21])
edge_attr:  torch.Size([12605, 21])
